In [14]:
import pandas as pd

import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_percentage_error
import pickle
from sktime.forecasting.model_selection import temporal_train_test_split

In [39]:
print(df.head(5))

   Unnamed: 0  stationId status  bikeStands  bikeNum   fetchTime weather  \
0           0         95   OPEN          28       12  1708358841  Clouds   
1           1         65   OPEN          15       25  1708358850  Clouds   
2           2         39   OPEN           8       12  1708358858  Clouds   
3           3         20   OPEN          29        1  1708358860  Clouds   
4           4          5   OPEN          14       26  1708359059  Clouds   

     description icon  temperature  pressure  humidity  windSpeed  windDeg  \
0  broken clouds  04d       284.87    1029.0      73.0       5.14    260.0   
1  broken clouds  04d       284.87    1029.0      73.0       5.14    260.0   
2  broken clouds  04d       284.87    1029.0      73.0       5.14    260.0   
3  broken clouds  04d       284.87    1029.0      73.0       5.14    260.0   
4  broken clouds  04d       284.87    1029.0      73.0       5.14    260.0   

   windGust  visibility  
0       0.0     10000.0  
1       0.0     10000.

In [40]:
df.dtypes

Unnamed: 0       int64
stationId        int64
status          object
bikeStands       int64
bikeNum          int64
fetchTime        int64
weather         object
description     object
icon            object
temperature    float64
pressure       float64
humidity       float64
windSpeed      float64
windDeg        float64
windGust       float64
visibility     float64
dtype: object

In [41]:
cardinality = df.nunique()
print("Cardinality of Columns:")
print(cardinality)

Cardinality of Columns:
Unnamed: 0     1134900
stationId          114
status               1
bikeStands          41
bikeNum             41
fetchTime       152835
weather              7
description         17
icon                15
temperature       1219
pressure            62
humidity            42
windSpeed           58
windDeg             92
windGust             1
visibility          23
dtype: int64


drop unnamed,status,windgust,icon 

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134900 entries, 0 to 1134899
Data columns (total 16 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Unnamed: 0   1134900 non-null  int64  
 1   stationId    1134900 non-null  int64  
 2   status       1134900 non-null  object 
 3   bikeStands   1134900 non-null  int64  
 4   bikeNum      1134900 non-null  int64  
 5   fetchTime    1134900 non-null  int64  
 6   weather      1134900 non-null  object 
 7   description  1134900 non-null  object 
 8   icon         1134900 non-null  object 
 9   temperature  1134900 non-null  float64
 10  pressure     1134900 non-null  float64
 11  humidity     1134900 non-null  float64
 12  windSpeed    1134900 non-null  float64
 13  windDeg      1134900 non-null  float64
 14  windGust     178875 non-null   float64
 15  visibility   1134900 non-null  float64
dtypes: float64(7), int64(5), object(4)
memory usage: 138.5+ MB


In [43]:
columns_to_drop = ["Unnamed: 0", "status", "windGust", "icon",]
df.drop(columns=columns_to_drop, inplace=True)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134900 entries, 0 to 1134899
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   stationId    1134900 non-null  int64  
 1   bikeStands   1134900 non-null  int64  
 2   bikeNum      1134900 non-null  int64  
 3   fetchTime    1134900 non-null  int64  
 4   weather      1134900 non-null  object 
 5   description  1134900 non-null  object 
 6   temperature  1134900 non-null  float64
 7   pressure     1134900 non-null  float64
 8   humidity     1134900 non-null  float64
 9   windSpeed    1134900 non-null  float64
 10  windDeg      1134900 non-null  float64
 11  visibility   1134900 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 103.9+ MB


In [45]:
#show the correlation as percentage
df[['bikeNum','temperature','pressure','humidity','windSpeed','windDeg','visibility']].corr() * 100

,bikeNum,temperature,pressure,humidity,windSpeed,windDeg,visibility
bikeNum,100.000000,-2.903127,-0.737736,0.521370,-0.498515,-0.426896,0.051923
temperature,-2.903127,100.000000,24.326374,-24.263483,23.112481,11.915596,6.588138
pressure,-0.737736,24.326374,100.000000,-27.963535,-14.226566,20.776119,17.279151
humidity,0.521370,-24.263483,-27.963535,100.000000,-26.975123,-25.305371,-36.218773
windSpeed,-0.498515,23.112481,-14.226566,-26.975123,100.000000,4.140900,4.721742
windDeg,-0.426896,11.915596,20.776119,-25.305371,4.140900,100.000000,37.038528
visibility,0.051923,6.588138,17.279151,-36.218773,4.721742,37.038528,100.000000


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134900 entries, 0 to 1134899
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   stationId    1134900 non-null  int64  
 1   bikeStands   1134900 non-null  int64  
 2   bikeNum      1134900 non-null  int64  
 3   fetchTime    1134900 non-null  int64  
 4   weather      1134900 non-null  object 
 5   description  1134900 non-null  object 
 6   temperature  1134900 non-null  float64
 7   pressure     1134900 non-null  float64
 8   humidity     1134900 non-null  float64
 9   windSpeed    1134900 non-null  float64
 10  windDeg      1134900 non-null  float64
 11  visibility   1134900 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 103.9+ MB


In [47]:
df.nunique()

stationId         114
bikeStands         41
bikeNum            41
fetchTime      152835
weather             7
description        17
temperature      1219
pressure           62
humidity           42
windSpeed          58
windDeg            92
visibility         23
dtype: int64

In [48]:
df['fetchTime'] = pd.to_datetime(df['fetchTime'], unit='s')

# Round the fetchTime to the nearest 5 minutes
df['fetchTime'] = df['fetchTime'].dt.round('5min')

In [49]:
df

,stationId,bikeStands,bikeNum,fetchTime,weather,description,temperature,pressure,humidity,windSpeed,windDeg,visibility
0,95,28,12,2024-02-19 16:05:00,Clouds,broken clouds,284.87,1029.0,73.0,5.14,260.0,10000.0
1,65,15,25,2024-02-19 16:10:00,Clouds,broken clouds,284.87,1029.0,73.0,5.14,260.0,10000.0
2,39,8,12,2024-02-19 16:10:00,Clouds,broken clouds,284.87,1029.0,73.0,5.14,260.0,10000.0
3,20,29,1,2024-02-19 16:10:00,Clouds,broken clouds,284.87,1029.0,73.0,5.14,260.0,10000.0
4,5,14,26,2024-02-19 16:10:00,Clouds,broken clouds,284.87,1029.0,73.0,5.14,260.0,10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1134895,63,24,11,2024-03-27 16:35:00,Clouds,few clouds,281.33,971.0,72.0,7.20,180.0,10000.0
1134896,64,22,18,2024-03-27 16:35:00,Clouds,few clouds,281.33,971.0,72.0,7.20,180.0,10000.0
1134897,18,18,12,2024-03-27 16:35:00,Clouds,few clouds,281.33,971.0,72.0,7.20,180.0,10000.0
1134898,65,19,21,2024-03-27 16:35:00,Clouds,few clouds,281.33,971.0,72.0,7.20,180.0,10000.0


In [50]:
df = pd.get_dummies(df, columns=['weather', 'description'])

df['stationId'] = df['stationId'].astype('int64')

print(df.dtypes)

stationId                                            int64
bikeStands                                           int64
bikeNum                                              int64
fetchTime                                   datetime64[ns]
temperature                                        float64
pressure                                           float64
humidity                                           float64
windSpeed                                          float64
windDeg                                            float64
visibility                                         float64
weather_Clear                                         bool
weather_Clouds                                        bool
weather_Drizzle                                       bool
weather_Fog                                           bool
weather_Mist                                          bool
weather_Rain                                          bool
weather_Snow                                          bo

In [15]:
df.iloc[0,:].to_csv('cleaned_data_sample.csv')

In [25]:
def data_cleaning(df, stationId):
    # stationId = 1
    df = df[df['stationId'] == stationId]
    total_capacity = list(df['bikeStands'])[0] + list(df['bikeNum'])[0]

    df['fetchTime'] = pd.to_datetime(df['fetchTime'])
    df = df.sort_values('fetchTime')

    # Create a date range that covers all dates in your DataFrame, at 5 minute frequency
    date_range = pd.date_range(start=df['fetchTime'].min(), end=df['fetchTime'].max(), freq='5T')

    duplicates = df[df.duplicated('fetchTime', keep=False)]
    df = df.drop_duplicates('fetchTime', keep='first')

    df_complete = df.set_index('fetchTime').reindex(date_range, method='ffill').reset_index()

    object_columns = df_complete.select_dtypes(['object']).columns
    for col in object_columns:
        df_complete[col] = df_complete[col].astype('category')

    df_complete = pd.get_dummies(df_complete, columns=['description'])
    df_complete = df_complete.drop(['stationId', 'bikeStands','weather'], axis=1)

    train, test = temporal_train_test_split(df_complete, test_size=0.3)
    y_train, x_train = train['bikeNum'], train.drop(['bikeNum'], axis=1)
    y_test, x_test = test['bikeNum'], test.drop(['bikeNum'], axis=1)

    x_train, x_test = x_train.set_index('index'), x_test.set_index('index')
    y_train, y_test = pd.DataFrame(y_train).set_index(x_train.index), pd.DataFrame(y_test).set_index(x_test.index)
    return x_train, y_train, x_test, y_test, total_capacity

In [26]:
df = pd.read_csv('C:/Users/31003/Desktop/UCD/comp30830/new_cleaned_toTrainData.csv')
all_stationId = list(set(df['stationId']))

In [27]:
 x_train, y_train, x_test, y_test, total_capacity = data_cleaning(df,1)

C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fetchTime'] = pd.to_datetime(df['fetchTime'])


In [89]:
# We need to group by stationId to train by each stationId
# grouped = df_sorted.groupby('stationId')

# import pandas as pd

# df_sorted = df.sort_values(by='fetchTime', ascending=True)

# # Group by 'stationId'
# grouped = df_sorted.groupby('stationId')
#print(grouped)
# for stationId, group in grouped:
#testing only on 70% of data
def sklearn_regress(stationId,X_train,X_test,y_train,y_test,total_capacity):
    # Linear Regression model
    linear_model = LinearRegression()
    linear_model.fit(X_train, y_train)
    
    # Ridge model
    ridge_model = Ridge(alpha=10)
    ridge_model.fit(X_train, y_train)
    
    # Lasso model
    lasso_model = Lasso(alpha=0.01)
    lasso_model.fit(X_train, y_train)

    model_situation = {}

    models = [linear_model, ridge_model, lasso_model]
    model_name = ['linear_regression','ridge_regression','lasso_regression']
    for i in [0,1,2]:
        print(i)
        model= models[i]
        name = model_name[i]
        y_pred = model.predict(X_test)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        print(f"Station {stationId}, Model {type(model).__name__}, MAPE: {mape}")
        anamaly = len([x for x in y_pred if x > total_capacity or x < 0])
        model_situation[name] = {'mape': mape, 'anamaly': anamaly,'model':name}
        
    # with open(f'linear_model_{stationId}.pkl', 'wb') as file:
    #     pickle.dump(linear_model, file)
    # with open(f'ridge_model_{stationId}.pkl', 'wb') as file:
    #     pickle.dump(ridge_model, file)
    # with open(f'lasso_model_{stationId}.pkl', 'wb') as file:
    #     pickle.dump(lasso_model, file)

    return model_situation


In [90]:
df = pd.read_csv('C:/Users/31003/Desktop/UCD/comp30830/new_cleaned_toTrainData.csv')
all_stationId = list(set(df['stationId']))
modelestimate_dic = {}
for stationId in all_stationId:
    print(f'{stationId} begins:')
    x_train,y_train,x_test,y_test,total_capacity = data_cleaning(df,stationId)
    model_situation = sklearn_regress(stationId,x_train,x_test,y_train,y_test,total_capacity)

    modelestimate_dic[stationId] = model_situation
    #df_modelestimate.to_csv(f'C:/Users/31003/Desktop/UCD/comp30830/station{stationId}_sktime_training_situation.csv')
    print()
    print()

1 begins:
0
Station 1, Model LinearRegression, MAPE: 4510653300926868.0
1
Station 1, Model Ridge, MAPE: 4548873817663092.0
2
Station 1, Model Lasso, MAPE: 4587687138159392.0


2 begins:
0
Station 2, Model LinearRegression, MAPE: 5600868785984865.0
1
Station 2, Model Ridge, MAPE: 5605672231637316.0
2
Station 2, Model Lasso, MAPE: 5617885704187954.0


3 begins:


C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fetchTime'] = pd.to_datetime(df['fetchTime'])
C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fetchTime'] = pd.to_datetime(df['fetchTime'])
C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

0
Station 3, Model LinearRegression, MAPE: 2059614514270669.5
1
Station 3, Model Ridge, MAPE: 2060705694697344.0
2
Station 3, Model Lasso, MAPE: 2073567601003202.5


4 begins:
0
Station 4, Model LinearRegression, MAPE: 549559760619826.2
1
Station 4, Model Ridge, MAPE: 549556718454958.56
2
Station 4, Model Lasso, MAPE: 550248057263983.9


5 begins:
0
Station 5, Model LinearRegression, MAPE: 8115832280458522.0
1
Station 5, Model Ridge, MAPE: 8149386345115281.0
2
Station 5, Model Lasso, MAPE: 8153753958402188.0


6 begins:


C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fetchTime'] = pd.to_datetime(df['fetchTime'])
C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fetchTime'] = pd.to_datetime(df['fetchTime'])
C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

0
Station 6, Model LinearRegression, MAPE: 5714857723588552.0
1
Station 6, Model Ridge, MAPE: 5731571540528787.0
2
Station 6, Model Lasso, MAPE: 5708657551947179.0


7 begins:
0
Station 7, Model LinearRegression, MAPE: 4250674328094158.0
1
Station 7, Model Ridge, MAPE: 4269151983165440.0
2
Station 7, Model Lasso, MAPE: 4266997127585557.5


8 begins:
0
Station 8, Model LinearRegression, MAPE: 1118139458968049.1
1
Station 8, Model Ridge, MAPE: 1109471845324718.4
2
Station 8, Model Lasso, MAPE: 1101092309528043.9


9 begins:


...

0
Station 111, Model LinearRegression, MAPE: 4014782512895941.5
1
Station 111, Model Ridge, MAPE: 4009530370112031.5
2
Station 111, Model Lasso, MAPE: 4007460157805935.0


112 begins:
0
Station 112, Model LinearRegression, MAPE: 6135950516688109.0
1
Station 112, Model Ridge, MAPE: 6143237706137514.0
2
Station 112, Model Lasso, MAPE: 6152677722194451.0


113 begins:
0
Station 113, Model LinearRegression, MAPE: 2879201176465833.5
1
Station 113, Model Ridge, MAPE: 2882145113728381.0
2


C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fetchTime'] = pd.to_datetime(df['fetchTime'])
C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fetchTime'] = pd.to_datetime(df['fetchTime'])
C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

Station 113, Model Lasso, MAPE: 2886785934021316.0


114 begins:
0
Station 114, Model LinearRegression, MAPE: 7003018078380101.0
1
Station 114, Model Ridge, MAPE: 7004297057296776.0
2
Station 114, Model Lasso, MAPE: 7042370257391297.0


115 begins:
0
Station 115, Model LinearRegression, MAPE: 1210923720683204.5
1
Station 115, Model Ridge, MAPE: 1211752238056826.2
2
Station 115, Model Lasso, MAPE: 1212468183317983.8


116 begins:


C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fetchTime'] = pd.to_datetime(df['fetchTime'])
C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fetchTime'] = pd.to_datetime(df['fetchTime'])
C:\Users\31003\AppData\Local\Temp\ipykernel_18316\4216843704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

0
Station 116, Model LinearRegression, MAPE: 8246928236859060.0
1
Station 116, Model Ridge, MAPE: 8266733106171877.0
2
Station 116, Model Lasso, MAPE: 8270124637731372.0


117 begins:
0
Station 117, Model LinearRegression, MAPE: 847012860691606.5
1
Station 117, Model Ridge, MAPE: 856301271411332.9
2
Station 117, Model Lasso, MAPE: 866827150306207.1




In [94]:
linear_mape,ridge_mape,lasso_mape = 0,0,0
linear_anamoly,ridge_anamoly,lasso_anamoly = 0,0,0
for i in modelestimate_dic:
    d = modelestimate_dic[i]
    
    linear_mape += d['linear_regression']['mape']
    ridge_mape += d['ridge_regression']['mape']
    lasso_mape += d['lasso_regression']['mape']

    linear_anamoly += d['linear_regression']['anamaly']
    ridge_anamoly += d['ridge_regression']['anamaly']
    lasso_anamoly += d['lasso_regression']['anamaly']

In [97]:
print(linear_mape/114,ridge_mape/114,lasso_mape/114)
print(linear_anamoly/114,ridge_anamoly/114,lasso_anamoly/114)

3799739190931483.0 3800149036446643.5 3798259988628133.5
20.57894736842105 18.92105263157895 17.37719298245614
